In [1]:
import sys
sys.path.append("../..")

In [ ]:
from prettytable import PrettyTable, ALL
from textwrap import fill
from msprof_analyze.advisor.interface.interface import Interface

In [2]:
profiling_path = "YOUR PROFILING PATH"
interface = Interface(profiling_path=profiling_path)

### 融合算子API识别

指定profiling路径后，可以自动识别其中包含的融合算子并给出对应的torch_npu api和需要修改的代码堆栈。基于给定堆栈可以快速定位到需要修改的代码段，替换torch_npu api后，能够减少pytorch侧的小算子的下发，进而提升模型训练速度。

In [3]:
timeline_fusion_ops_result = interface.get_result("schedule", "timeline_fusion_ops")

In [4]:
display_column_num = 3
problems = timeline_fusion_ops_result.get("problems")
problem_table = PrettyTable(problems.get("headers")[:display_column_num])
for row in problems.get("data"):
    for i in range(len(row)):
        row[i] = fill(str(row[i]), width=80)
    problem_table.add_row(row[:display_column_num])

display(problem_table)

problem,description,suggestion
timeline_fusion_ops,Found 2 apis to be replaced based on the runtime env cann-8.0.RC1 and torch-2.1.0,1. Please replace training api according to sub table 'Affinity training api'


如下所示，存在亲和优化器和梯度裁剪两个可替换的torch_npu api，并给出了具体的堆栈。

In [5]:
fusion_ops_api = timeline_fusion_ops_result.get("timeline_fusion_ops")
if fusion_ops_api:
    fusion_ops_api_table = PrettyTable(fusion_ops_api.get("headers"))

    for row in fusion_ops_api.get("data"):
        for i in range(len(row)):
            row[i] = fill(str(row[i]), width=80)
        fusion_ops_api_table.add_row(row)

    fusion_ops_api_table.hrules = ALL
    display(fusion_ops_api_table)

Affinity API,Code stacks,Stack called counts
optimizer.clip_grad_norm_fused_,/home/ma-user/anaconda3/envs/PyTorch-1.11.0/lib/python3.9/site-packages/torch/nn/utils/clip_grad.py(49): clip_grad_norm_; /home/ma-user/work/algorithms/doc_cls/Bert.py(205): train_epoch; /home/ma-user/work/algorithms/doc_cls/Bert.py(252): <module>,2
torch_npu.optim.NpuFusedAdamW,/home/ma-user/anaconda3/envs/PyTorch-1.11.0/lib/python3.9/site-packages/torch_npu/npu/profiler.py(675): __enter__; /home/ma-user/anaconda3/envs/PyTorch-1.11.0/lib/python3.9/site-packages/torch_npu/npu/profiler.py(719): wrapper; /home/ma-user/anaconda3/envs/PyTorch-1.11.0/lib/python3.9/site-packages/torch/optim/lr_scheduler.py(65): wrapper; /home/ma-user/work/algorithms/doc_cls/Bert.py(219): train_epoch; /home/ma-user/work/algorithms/doc_cls/Bert.py(252): <module>,2
